# Explosion detector

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
plt.rc('axes', axisbelow=True)
plt.rc('image', cmap='magma_r')
np.random.seed(42)

Define the coordinate system

In [ ]:
NUM_POINTS = 5000
LAPS = 50

radius = np.arange(1, NUM_POINTS + 1) / NUM_POINTS
theta = LAPS * 2 * np.pi * radius
spiral_x = radius * np.cos(theta)
spiral_y = radius * np.sin(theta)
spiral_locations = np.column_stack((spiral_x, spiral_y))

Define the locations of the detection stations on the surface:

In [ ]:
NUM_SENSORS = 10
sensors = np.arange(1, NUM_SENSORS + 1)
sensors_theta = 2 * np.pi * sensors / NUM_SENSORS
sensors_x = np.cos(sensors_theta)
sensors_y = np.sin(sensors_theta)
sensors_locations = np.column_stack((sensors_x, sensors_y))

Plot all locations:

In [ ]:
def plot_earth(spiral_locations, sensors_thetas,
               colors=None, sensors_values=None, alpha=0.15,
               spiral_markers_size=2, sensors_markers_size=50,
               figure_size=8, delta_degrees=1):
    plt.figure(figsize=(figure_size, figure_size))
    plt.axis('equal')
    plt.grid()

    # Spiral locations
    spiral_x, spiral_y = spiral_locations.T
    plt.plot(spiral_x, spiral_y, alpha=alpha)
    spiral_kwargs = {}
    if colors is not None:
        spiral_kwargs['c'] = colors
    plt.scatter(spiral_x, spiral_y, spiral_markers_size, **spiral_kwargs)
    if colors is not None:
        plt.colorbar()

    # Stations
    sensors_x = np.cos(sensors_theta)
    sensors_y = np.sin(sensors_theta)
    delta_theta = np.radians(delta_degrees)
    plt.scatter(sensors_x, sensors_y, sensors_markers_size, label='Sensors')
    if sensors_values is not None:
        clean, noisy = sensors_values
        clean_thetas = sensors_theta - delta_theta
        noisy_thetas = sensors_theta + delta_theta
        # TODO

    plt.legend()

In [ ]:
plot_earth(spiral_locations, sensors_locations)

Define what value on each sensor would be generated by an explosion at internal spiral location:

In [ ]:
def detected_values(earthquake_locations, sensor_location):
    diff = earthquake_locations - sensor_location
    d = np.linalg.norm(diff, axis=1)
    measured = 1 / (d**2 + 0.1)
    return measured

In [ ]:
v = np.empty((NUM_POINTS, NUM_SENSORS))
for i, sensor_location in enumerate(sensors_locations):
    v[:, i] = detected_values(spiral_locations, sensor_location)

Make the explosion data:

In [ ]:
true_earthquake_idx = np.random.choice(len(spiral_locations))
true_earthquake_location = spiral_locations[true_earthquake_idx]

Standard deviation $\sigma$ of the Gaussian noise:

In [ ]:
sigma = 1

Get the noisy sensor values that will be observed for this explosion:

In [ ]:
values_clean = detected_values(true_earthquake_location, sensors_locations)
values_noisy = values_clean + sigma * np.random.randn(NUM_SENSORS)

In [ ]:
plt.figure()

plt.plot(values_clean, 'o', label='Theoretical')
plt.plot(values_noisy, 'o', label='Observed')
plt.xticks(range(NUM_SENSORS))
plt.xlabel('Sensor')
plt.ylabel('$v_i$')

plt.grid()
plt.legend()
plt.show()

Perform inference $p(\text{location}|\text{observed sensor values})$ given these sensor values:

In [ ]:
# This is not optimised :(
logp = np.zeros((NUM_POINTS, 1))
for spiral_idx in range(NUM_POINTS):
    for sensor_idx in range(NUM_SENSORS):
        # Gaussian distribution
        logp[spiral_idx] += -0.5 * (values_noisy[sensor_idx] - v[spiral_idx, sensor_idx])**2 / (sigma**2)
p = np.exp(logp - logp.max()) # do exponentiation (and avoid over/underflow)
p /= p.sum() # normalise

Plot the posterior and most likely location of the explosion:

In [ ]:
p_max_index = np.argmax(p)
colors = p / p.max()
colors = colors.ravel()
plot_earth(spiral_locations, sensors_locations,
           colors=colors, alpha=0.05, spiral_markers_size=10,
           sensors_values=(values_clean, values_noisy))